# SPRINT 8. Parte II
> PowerBI + Python

## Nivel 1
Pasar todos los gráficos del S8_1N1 a PowerBI.

El Script de conexión e importación de datos es este:

In [ ]:
import dateutil
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import configparser
import mysql.connector

cfgFp = open(r'C:\Users\formacio\Downloads\mlg\git_repo\SPRINT8\db.ini', mode='r')
iniFp = configparser.ConfigParser()
iniFp.read_file(cfgFp)
dbCfg = dict(iniFp.items('database'))

# exit()

db = mysql.connector.connect(
    host=dbCfg['host'],
    database=dbCfg['database'],
    user=dbCfg['user'],
    password=dbCfg['password']
)

tr_df   = pd.read_sql("SELECT * FROM `transaction`", db)

user_df = pd.read_sql("SELECT * FROM `user`", db)

co_df = pd.read_sql("SELECT * FROM company", db)

pr_df   = pd.read_sql("SELECT * FROM product", db)
prtr_df = pd.read_sql("SELECT * FROM product_transaction", db)

def age(birthdate):
    dob = pd.to_datetime(birthdate).date() if type(birthdate) is not str \
                  else dateutil.parser.parse(birthdate).date()
    hoy = dateutil.utils.today().date()
    ddiff = hoy - dob
    edad  = int(ddiff.days / 365.25)
    return edad

# Add age column to user dataframe
user_df["age"] = user_df['birthdate'].map(age)


### S8.2N1E1
Distribución de edades de los usuarios. El script utilizado es:

In [ ]:
import matplotlib.pyplot as plt

user_df = dataset
user_df.groupby('age')['age'] \
        .count() \
        .plot.bar(
            x='age',
            # figsize=(10,5),
            title="Age distribution",
            ylabel="# of Users",
            xlabel="Age"
        )
plt.show()

In [ ]:
# Y el histograma es este. Aún no estoy seguro si dejarlo.


### S8.1N1E2
Dos variables numéricas.
El código adaptado ha quedado así:

In [ ]:
from matplotlib import pyplot, ticker
import numpy as np

productos = dataset
salesXage = productos.get(['age', 'amount']) \
                     .groupby('age')['amount'] \
                     .agg(['sum', 'count'])

f, sxa_gr = pyplot.subplots(figsize=(12,8))
ages_list = salesXage.index
bar_width = 0.4
positions = np.arange(len(ages_list))

sxa_gr.bar(
    x=positions,
    height=salesXage['count'], # transactions by age
    width=bar_width,
    label='# of sales',
    color='#5c5'
)

sxa_gr.set_xlabel('Age')
sxa_gr.set_ylabel('# of transactions')
sxa_gr.set_xticks(positions, [str(n) for n in ages_list])

sxa_gr2 = sxa_gr.twinx()
sxa_gr2.bar(
    x=positions+bar_width,
    height=salesXage['sum'] / 1000,
    width=bar_width,
    label='Sales in M€',
    color='#393'
)
sxa_gr2.set_ylabel('Sales in (in thousands of €)')
sxa_gr2.yaxis.set_major_formatter(ticker.FormatStrFormatter('%dM €'))

sxa_gr.set_title("Sales and revenue by customer age")
sxa_gr.legend(loc='upper left')
sxa_gr2.legend(loc='upper right')

pyplot.show()

In [32]:
# Prolog - Auto Generated #
import os
import pandas
import matplotlib.pyplot as plt
import sys

sys.tracebacklimit = 0

os.chdir(u'C:/Users/formacio/PythonEditorWrapper_536b409e-c951-4a91-a685-7f8c52129589')
dataset = pandas.read_csv('input_df_61b33d10-284a-4db5-96a7-85045da7bfd6.csv')

# matplotlib.pyplot.figure(figsize=(5.55555555555556,4.16666666666667), dpi=72)
# matplotlib.pyplot.show = lambda args=None,kw=None: matplotlib.pyplot.savefig(str(uuid.uuid1()))
# Original Script. Please update your script content here and once completed copy below section back to the original editing window #
# El código siguiente, que crea un dataframe y quita las filas duplicadas, siempre se ejecuta y actúa como un preámbulo del script: 

# dataset = pandas.DataFrame(amount, declined, transaction_id, transaction_ts)
# dataset = dataset.drop_duplicates()

# Pegue o escriba aquí el código de script:
def column_colors(values, color_map: dict):
    cc = []
    for n in values:
        cc.append(get_color(n, color_map))
    return cc

def get_color(number, color_map):
    sorted_keys = sorted(key for key in color_map.keys() if isinstance(key, int))

    # Primero los valores extremos...
    if number < sorted_keys[0]:
        return color_map[sorted_keys[0]]
    elif number > sorted_keys[-1]:
        return color_map['else']

    # Ahora los valores de las keys
    for key in sorted_keys:
        if number < key:
            return color_map[key]

    return color_map[sorted_keys[-1]]

tr_df = dataset

tr_df_agg  = tr_df[tr_df['declined']==0]  \
                .groupby(["Año", "Mes"], ) \
                .agg(['sum', 'count'])
tr_agg     = tr_df_agg['amount']
bar_values = tr_agg['count'].to_list()
print("Transactions Aggregated\n", tr_agg)
semaphor_colors = {
    20:     '#f00',
    40:     '#fb0',
    'else': '#0f0'
}
cc = column_colors(bar_values, semaphor_colors)
dates = tr_agg.index
xticks = [f"{d[0]} - {d[1]}" for d in tr_agg.T]

print(xticks)

# No he visto una forma de "desindexar" o "aplanar" la serie,
# así que he tirado de bucles.
mil = tr_agg.index.levels # multiindexes. 0=month, 1=DOW, 2=hour
fg_df_flat = []
for y in mil[0]:
    for m in mil[1]:
        print(f"Index: {y} - {m}", tr_agg[(y,m)])
        fg_df_flat.append([
            y,
            m,
            tr_agg[(y,m)]
                if (y,m) in tr_agg.keys()
                else 0
        ])

print(fg_df_flat)

graph = tr_agg['count'].plot.bar(
    # figsize=(9,6),
    # color=cc,
    rot=60,
    ylabel="# of transactions",
    xlabel="Month",
    label="Transactions"
)
graph.set_xticks(range(0, xticks.__len__()), xticks)
# graph.legend(["Transactions"])
graph.plot()

Transactions Aggregated
                       sum  count
Año  Mes                        
2021 abril       13507.68     54
     agosto      13900.24     50
     diciembre   14177.16     53
     julio        9931.77     43
     junio       10817.81     38
     marzo        5711.35     20
     mayo        11659.27     42
     noviembre    7675.74     30
     octubre     10765.57     41
     septiembre   9524.43     34
2022 enero        8236.31     37
     febrero     10077.55     39
     marzo        3520.71     19
['2021 - abril', '2021 - agosto', '2021 - diciembre', '2021 - julio', '2021 - junio', '2021 - marzo', '2021 - mayo', '2021 - noviembre', '2021 - octubre', '2021 - septiembre', '2022 - enero', '2022 - febrero', '2022 - marzo']


KeyError: (2021, 'abril')